In [57]:
import win32com.client
import subprocess
import psutil
import os
import time
import logging
import psycopg2
from psycopg2 import Binary
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [58]:
OUTLOOK_PATHS = [
    r"C:\Program Files\Microsoft Office\root\Office16\OUTLOOK.EXE",
    r"C:\Program Files (x86)\Microsoft Office\root\Office16\OUTLOOK.EXE",
]

# Verificando se o outlook está aberto antes de iniciar o processo
def outlook_is_open():
    for proc in psutil.process_iter(['name']):
        if proc.info['name'] and proc.info['name'].lower() == 'outlook.exe':
            return True
    return False

In [59]:
# Func para abrir o outlook via process
def open_outlook():
    for path in OUTLOOK_PATHS:
        if os.path.exists(path):
            subprocess.Popen([path])
            return True 
    raise FileNotFoundError("Outlook não se encontra em nenhum dos caminhos")

In [60]:
if not outlook_is_open():
    logger.info('Iniciando abertura do outlook')
    open_outlook()
    time.sleep(30)
else:
    outlook = win32com.client.Dispatch("Outlook.Application")
    logger.info('Outlook aberto corretamente')

2025-12-22 16:57:43,416 - INFO - Outlook aberto corretamente


In [61]:
outlook = win32com.client.Dispatch("Outlook.Application")

Principais IDs de pastas:

6 → Inbox

5 → Sent Items

3 → Deleted Items

4 → Outbox

In [62]:
namespace = outlook.GetNamespace("MAPI")
inbox = namespace.GetDefaultFolder(6)  # 6 = Inbox
messages = inbox.Items

In [63]:
# Criando pasta no outlook
folder_name = "Processados"
status_folder = False

for i, folder in enumerate(inbox.Folders):
    print(i, folder.name)
    if folder.name == folder_name:
        logger.info(f"Pasta já existe")
        break 

0 Trash
1 Sent
2 Drafts
3 Junk
4 Processeds


In [64]:
# project_path = os.path.abspath(__file__)
project_path = os.path.dirname(os.path.abspath("."))
project_path

'c:\\Users\\felip\\Projects\\Gbpa\\celebre\\outlookRpa'

In [72]:
messages.Restrict("[Unread] = true").Count

0

In [65]:
project_path = os.path.dirname(os.path.abspath("."))
download_folder = os.path.join(project_path, 'tmp')

os.makedirs(download_folder, exist_ok=True)

logger.info(f"Pasta de download configurada para: {download_folder}")

messages = inbox.Items.Restrict("[Unread] = true")
messages.Sort("[ReceivedTime]", True)

pdf_attachments = []

for msg in messages:
    try:
        subject = msg.Subject
        sender_email = msg.SenderEmailAddress
        received_time = msg.ReceivedTime

        total_attachments = msg.Attachments.Count
        logger.info(f"Email de {sender_email} veio com {total_attachments} anexos")

        for i in range(1, total_attachments + 1):
            attachment = msg.Attachments.Item(i)
            filename = attachment.FileName or ""

            if not filename.lower().endswith(".pdf"):
                continue

            logger.info(f"Tratando o email {subject} enviado por {sender_email}")

            file_path = os.path.join(download_folder, filename)
            attachment.SaveAsFile(file_path)

            pdf_attachments.append({
                "email_subject": subject,
                "sender": sender_email,
                "received_time": msg.ReceivedTime.strftime("%Y-%m-%d %H:%M:%S"),
                "filename": filename,
                "file_path": file_path
            })

    except Exception as e:
        logger.error(f"Erro ao processar email '{subject}': {e}")

2025-12-22 16:57:43,506 - INFO - Pasta de download configurada para: c:\Users\felip\Projects\Gbpa\celebre\outlookRpa\tmp


In [66]:
for pdf in pdf_attachments:
    with open(pdf["file_path"], "rb") as f:
        pdf["content_bytes"] = f.read()

pdf_attachments

[]

In [67]:
load_dotenv()

database = os.getenv("POSTGRES_DB")
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")

conn = psycopg2.connect(
    host="localhost",
    database=database,
    user=user,
    password=password,
    port=5432
)

In [68]:
cursor = conn.cursor()

In [69]:
query = """
INSERT INTO broker_emails (
    email_subject,
    sender_email,
    received_time,
    filename,
    file_data
) VALUES (%s, %s, %s, %s, %s)
"""

for pdf in pdf_attachments:
    cursor.execute(
        query,
        (
            pdf["email_subject"],
            pdf["sender"],
            pdf["received_time"],
            pdf["filename"],
            Binary(pdf["content_bytes"])
        )
    )

conn.commit()


In [70]:
cursor.close()
conn.close()